In [1]:
#importing packages for data manuplation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#for easy access to path
import sys
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join('data')))
sys.path.insert(0,'../scripts_/')

In [3]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_column', None)
pd.set_option('max_rows', None)

df = pd.read_csv('../data/AdSmartABdata.csv')
df.head()

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,2020-07-10,8,Generic Smartphone,6,Chrome Mobile,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,2020-07-07,10,Generic Smartphone,6,Chrome Mobile,0,0
2,0016d14a-ae18-4a02-a204-6ba53b52f2ed,exposed,2020-07-05,2,E5823,6,Chrome Mobile WebView,0,1
3,00187412-2932-4542-a8ef-3633901c98d9,control,2020-07-03,15,Samsung SM-A705FN,6,Facebook,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0


### Data Preparation

In [6]:
df_test = df[['auction_id', 'experiment', 'yes', 'no']]
not_ans_df = cleaned_df.query("yes == 0 & no == 0")
not_ans_df.head()

,auction_id,experiment,yes,no
0,0008ef63-77a7-448b-bd1e-075f42c55e39,exposed,0,0
1,000eabc5-17ce-4137-8efe-44734d914446,exposed,0,0
3,00187412-2932-4542-a8ef-3633901c98d9,control,0,0
4,001a7785-d3fe-4e11-a344-c8735acacc2c,control,0,0
5,0027ce48-d3c6-4935-bb12-dfb5d5627857,control,0,0


In [10]:
df_test = df_test.query("not (yes == 0 & no == 0)")
df_test.groupby("experiment").agg({"yes": "sum", "no":"sum"})

df_test_agg = df_test.groupby("experiment").agg({"yes": "sum", "no":"sum"})
df_test_agg['aware'] = [0,0]
df_test_agg['aware']['control'] = df_test_agg['yes']['control']
df_test_agg['aware']['exposed'] = df_test_agg['yes']['exposed']

df_test_agg['total'] = [0, 0]
df_test_agg['total']['control'] = df_test_agg['yes']['control'] +  df_test_agg['no']['control']
df_test_agg['total']['exposed'] = df_test_agg['yes']['exposed'] + df_test_agg['no']['exposed']

print(f"number of pepole that are aware of the brand lux in control group are: {df_test_agg['aware']['control']}")
print(f"number of pepole that are aware of the brand lux in exposed group are: {df_test_agg['aware']['exposed']}")

test_df = df_test_agg.copy()
test_df

number of pepole that are aware of the brand lux in control group are: 264
number of pepole that are aware of the brand lux in exposed group are: 308


,yes,no,aware,total
experiment,,,,
control,264,322,264,586
exposed,308,349,308,657
